<a href="https://colab.research.google.com/github/TienLungSun/LLM/blob/main/%E7%AF%84%E4%BE%8B_Fine_tune_Taiwan_LLaMa_2_13b_in_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tune Taiwan-LLaMa 2 in Google Colab

# 先備知識

[參考文章] 為什麼不 fine tune 模型?

在做 computer vision 的年代，我們會用 fine tune 的技巧，把自己的資料餵給 CNN（Convolutional Neural Networks），fine tune 模型而得到一個新的模型。而且準確度往往很容易做得很高，細心調整的話，達到 90%以上往往不是一件難事。但是在 LLM 的時代就不太一樣了，如果我們使用 fine tune 的技巧來把自己想要的資料訓練進去，例如說員工手冊等，效果往往不盡人意。也因此才發展出使用 RAG 的手法。那麼在什麼情況下 fine tune 是會比較好的呢？Open AI 給出下列的情境，大家可以參考看看。

- 提高可操控性： fine-tune 使模型更好地遵循指示，例如使輸出簡潔或始終以特定語言回應。舉例來說，開發人員可以使用 fine-tune 來確保模型在使用德語時總是以德語回應。

- 可靠的輸出格式： fine-tune 改善了模型始終以一致的格式進行回應的能力，這對於需要特定響應格式的應用程式（例如幫忙寫程式碼，或構構 API 呼叫）至關重要。開發人員可以使用 fine-tune 更可靠地將用戶提示轉換為高品質的 JSON 片段，可與其自己的系統一起使用。

- 自定義語氣： fine-tune 是用來調整模型輸出的質感，例如語氣，使其更符合企業品牌的聲音。有著可識別品牌聲音的企業可以使用 fine-tune 使模型的語氣更加一致。

如果你的需求是讀企業內部的文件來做問答，例如說員工手冊、請假規則等的，就不建議使用 fine tune 的方式來達成，而會使用 RAG 的方式來完成。

參考來源: https://ithelp.ithome.com.tw/m/articles/10335727

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 104.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00


因為Taiwan-LLaMa2這個版本是13b參數大小的版本, 所以請確保你的GPU有V100(16GB)或A100(40GB佳!)的硬體規格

In [ ]:
!nvidia-smi

Fri Oct 20 05:53:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    22W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 載入相關套件

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

## 預訓練模型參數設定

In [ ]:
# Used for multi-gpu
local_rank = -1
per_device_train_batch_size = 4
per_device_eval_batch_size = 1
gradient_accumulation_steps = 4
learning_rate = 2e-4
max_grad_norm = 0.3
weight_decay = 0.001
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64
max_seq_length = 512

# The model that you want to train from the Hugging Face hub
model_name = "yentinglin/Taiwan-LLaMa-v1.0"

# Fine-tuned model name
new_model = "TW-llama-2-13b-beta0.1" #訓練後的新模型名稱

# The instruction dataset to use
#訓練的中文資料集在huggingface上面有很多，自己找囉!
#要留意資料集欄位的名稱，SFTTrainer設定時要提供指定的欄位名稱
dataset_name = "erhwenkuo/moss-003-sft-chinese-zhtw"

# Activate 4-bit precision base model loading
use_4bit = True

# Activate nested quantization for 4-bit base models
use_nested_quant = False

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4=
bnb_4bit_quant_type = "nf4"

# Number of training epochs
num_train_epochs = 1

# Enable fp16 training
fp16 = False

# Enable bf16 training
bf16 = False #colab A100 supported! (如果沒有A100-GPU記得要關掉!!)

# Use packing dataset creating
packing = False

# Enable gradient checkpointing
gradient_checkpointing = True

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine, and has advantage for analysis)
lr_scheduler_type = "constant"

# Number of optimizer update steps
max_steps = 10 #10000  ##記得要設定訓練回合數!! 教學範例只設10個steps示意一下

# Fraction of steps to do a warmup for
warmup_ratio = 0.03

# Group sequences into batches with same length (saves memory and speeds up training considerably)
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 10

# Log every X updates steps
logging_steps = 10

# The output directory where the model predictions and checkpoints will be written
output_dir = "./results"

# Load the entire model on the GPU 0
device_map = {"": 0}

### 載入資料集

2023/10/20 (更新) 一些HF平台上的繁體中文資料集:
- Taiwan-LLaMa模型訓練資料集:(https://huggingface.co/datasets/yentinglin/traditional_mandarin_instructions)
- Erhwen,Kuo個人整理,一些知名的語料集之繁體中文化:(https://huggingface.co/erhwenkuo)
- 網頁新聞,由簡體轉繁體(文化對齊問題請自行評估):(https://huggingface.co/datasets/jed351/Traditional-Chinese-Common-Crawl-Filtered)
- TAIDE計畫的測試資料集:(https://huggingface.co/datasets/taide/TAIDE-14-tasks)

(參考)簡體中文資料集:
- 北京智源人工智能研究院-BAAI DataHub（data.baai.ac.cn）:(https://huggingface.co/datasets/BAAI/COIG-PC)

(參考) github平台上的資源:
- 聯發創新基地（MediaTek Research)(https://github.com/mtkresearch/MR-Models/tree/main)

In [ ]:
dataset = load_dataset(dataset_name, split="train", num_proc=8) #colab有支援8個CPU,全用,加速載入

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1074551 [00:00<?, ? examples/s]

### 調整資料集格式,以符合SFTTrainer使用

In [ ]:
dataset.column_names, len(dataset['conversation'])

(['conversation_id', 'category', 'conversation'], 1074551)

In [ ]:
#調整資料集格式,以符合SFTTrainer使用 (來源資料集有107萬筆資料)
sub_dataset = dataset['conversation'][:10000][0] #使用的資料筆數,可自行調整,如果只是示範,資料筆數只需少數幾筆即可,不用全部取用

In [ ]:
type(sub_dataset)

list

In [ ]:
sub_dataset[0]

{'human': '如何保障工作中遵循正確的安全準則？',
 'assistant': '為了保障工作中遵循正確的安全準則，以下是一些建議：\n\n1.瞭解相關安全規定和標準：瞭解相關的安全規定和標準，並遵守它們。這可以包括公司和政府的安全標準，以及行業標準和最佳實踐。\n\n2.培訓和教育：確保您和您的同事接受了必要的培訓和教育，以瞭解正確的安全準則和行為。\n\n3.使用正確的工具和裝置：確保您使用正確的工具和裝置，並且它們得到了正確的維護和保養。\n\n4.個人防護裝備：確保您和您的同事穿戴正確的個人防護裝備，如安全鞋、透明眼鏡或面罩、手套等。\n\n5.建立一個安全文化：鼓勵個人對安全的責任感和意識，並建立一個安全文化，使人們始終關注他們自己和他人的安全。\n\n6.持續監測和改進：持續監測和改進安全準則和程式，以確保它們保持最新，並適應變化的情況。\n\n這些措施可以幫助您和您的同事在工作中保障正確的安全準則，確保您的工作場所是一個安全的環境。'}

In [ ]:
sub_dataset[0]['human']

'如何保障工作中遵循正確的安全準則？'

In [ ]:
sub_dataset[0]['assistant']

'為了保障工作中遵循正確的安全準則，以下是一些建議：\n\n1.瞭解相關安全規定和標準：瞭解相關的安全規定和標準，並遵守它們。這可以包括公司和政府的安全標準，以及行業標準和最佳實踐。\n\n2.培訓和教育：確保您和您的同事接受了必要的培訓和教育，以瞭解正確的安全準則和行為。\n\n3.使用正確的工具和裝置：確保您使用正確的工具和裝置，並且它們得到了正確的維護和保養。\n\n4.個人防護裝備：確保您和您的同事穿戴正確的個人防護裝備，如安全鞋、透明眼鏡或面罩、手套等。\n\n5.建立一個安全文化：鼓勵個人對安全的責任感和意識，並建立一個安全文化，使人們始終關注他們自己和他人的安全。\n\n6.持續監測和改進：持續監測和改進安全準則和程式，以確保它們保持最新，並適應變化的情況。\n\n這些措施可以幫助您和您的同事在工作中保障正確的安全準則，確保您的工作場所是一個安全的環境。'

In [ ]:
from datasets import Dataset

# 將資料轉為我們指定的格式(單欄位,包括有### human及 ### assistant的角色對話)
formatted_data = [{
    'text': f'### human: {entry["human"]}\n\n### assistant: {entry["assistant"]}'
} for entry in sub_dataset]

# 使用 Hugging Face 的 datasets 建立格式調整後的資料集
formatted_dataset = Dataset.from_dict({'text': [item['text'] for item in formatted_data]})

### 設定訓練環境參數

In [ ]:
print("=" * 80)
print("Load tokenizer and model with QLoRA configuration...")
print("=" * 80)

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

#check GPU support bf16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16, you can accelerate training with the argument --bf16")
        print("=" * 80)

print("=" * 80)
print("Load AutoModelForCausalLM from pretrained model...")
print("=" * 80)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

print("=" * 80)
print("Load AutoTokenizer from pretrained model...")
print("=" * 80)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token


Load tokenizer and model with QLoRA configuration...
Load AutoModelForCausalLM from pretrained model...


(…)iwan-LLaMa-v1.0/resolve/main/config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

(…)esolve/main/pytorch_model.bin.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

...

Load AutoTokenizer from pretrained model...


(…)-v1.0/resolve/main/tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

(…)n-LLaMa-v1.0/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)1.0/resolve/main/special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

## 模型訓練

### SFT-Trainer設定
Supervised Fine-tuning Trainer API說明

[https://huggingface.co/docs/trl/main/en/sft_trainer](https://huggingface.co/docs/trl/main/en/sft_trainer)

In [ ]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_dataset, #確認資料集名稱
    peft_config=peft_config,
    dataset_text_field="text",
    #dataset_text_field="conversation", #記得要改成資料集的對應欄位名稱!!!
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [ ]:
#train model
# %%time
try:
  trainer.train()

except KeyboardInterrupt:
    print("KeyboardInterrupt")

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,0.338900


### 儲存模型參數

In [ ]:
#trainer.model.save_pretrained(output_dir)

model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

### 進行模型預測

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

In [ ]:
text = "你知道什麼是台灣馬路三寶嗎?"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=512, temperature=0.2)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

你知道什麼是台灣馬路三寶嗎? - 自由講場 - Baby Kingdom - 親子王國 香港 討論區
Baby Kingdom - 親子王國 香港 討論區 › 首頁 › 自由講場› 你知道什麼是台灣馬路三寶嗎?
自由講場 今日: 417|主題: 1035300
發表於 19-10-10 17:11 |顯示全部帖子
本帖最後由 寶貝兒的媽媽 於 19-10-10 17:12 編輯
台灣馬路三寶是指在台灣駕駛時，遇到的三種不可預測的行為：
1. 闖紅燈
2. 不遵守交通規則
3. 駕駛時使用手機
發表於 19-10-10 17:14 |顯示全部帖子
寶貝兒的媽媽 發表於 19-10-10 17:11
台灣馬路三寶是指在台灣駕駛時，遇到的三種不可預測的行為： ...
發表於 19-10-10 17:15 |顯示全部帖子
發表於 19-10-10 17:16 |顯示全部帖子
發表於 19-10-10 17:17 |顯


In [ ]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 5120, padding_idx=0)
        (layers): ModuleList(
          (0-39): 40 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=5120, out_features=5120, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=5120, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
              (



---



## 相關參考資料
- [MiuLab] Taiwan-LLaMa2 [https://github.com/MiuLab/Taiwan-LLaMa](https://github.com/MiuLab/Taiwan-LLaMa)

- [Huggingface] Taiwan-LLaMa-v1.0 [https://huggingface.co/yentinglin/Taiwan-LLaMa-v1.0](https://huggingface.co/yentinglin/Taiwan-LLaMa-v1.0)

- Huggingface的中文資料集 [https://huggingface.co/datasets?language=language:zh&sort=trending](https://huggingface.co/datasets?language=language:zh&sort=trending)

2023/10/20 (更新) 一些HF平台上的繁體中文資料集:
- Taiwan-LLaMa模型訓練資料集:(https://huggingface.co/datasets/yentinglin/traditional_mandarin_instructions)
- Erhwen,Kuo個人整理,一些知名的語料集之繁體中文化:(https://huggingface.co/erhwenkuo)
- 網頁新聞,由簡體轉繁體(文化對齊問題請自行評估):(https://huggingface.co/datasets/jed351/Traditional-Chinese-Common-Crawl-Filtered)
- TAIDE計畫的測試資料集:(https://huggingface.co/datasets/taide/TAIDE-14-tasks)

(參考)簡體中文資料集:
- 北京智源人工智能研究院-BAAI DataHub（data.baai.ac.cn）:(https://huggingface.co/datasets/BAAI/COIG-PC)

(參考) github平台上的資源:
- 聯發創新基地（MediaTek Research)(https://github.com/mtkresearch/MR-Models/tree/main)

**code reference:**

- code based on Younes Belkada's [GitHub Gist](https://gist.github.com/younesbelkada/9f7f75c94bdc1981c8ca5cc937d4a4da).

- Llama2 finetune 範例: [https://colab.research.google.com/drive/16SlGXLuBRB30clB0dCYAh3sqk0edKoFC?usp=sharing](
https://colab.research.google.com/drive/16SlGXLuBRB30clB0dCYAh3sqk0edKoFC?usp=sharing)
